<a href="https://colab.research.google.com/github/zhaizeyu/test/blob/master/train_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p '/root/.kaggle'
from google.colab import userdata

# 打开文件以写入，如果文件不存在则创建
with open('/root/.kaggle/kaggle.json','w') as file:
    f = {"username":userdata.get('KAGGLE_USERNAME'),"key":userdata.get('KAGGLE_KEY')}
    # 写入内容 'ccc' 到文件
    file.write('{"username":"zeyuzhai","key":"42dc12af61163cb7e7c0a849dd7e8f49"}')
!chmod 600 '/root/.kaggle/kaggle.json'

In [ ]:
!kaggle datasets download -d nbroad/gemma-rewrite-nbroad

In [ ]:
!unzip gemma-rewrite-nbroad.zip

In [ ]:
!pip install -qU accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-recover"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0

device_map = 'auto'

In [ ]:
import pandas as pd
train_data_path='nbroad-v1.csv'
ori_data = pd.read_csv(train_data_path, usecols=['id', 'original_text', 'rewrite_prompt', 'rewritten_text'])


In [ ]:
template = """
<s>[INST] <<SYS>>
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.
    You are trying to understand how the original essay was transformed into a new version.
    Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.
    Start directly with the prompt, that's all I need. Output should be only line ONLY.
<</SYS>>
    Original Essay:
    {original_text}

    Rewritten Essay:
    {rewritten_text}
 [/INST]
    {rewrite_prompt}
"""

In [ ]:
ori_data["promot"] = ori_data.apply(lambda row: template.format(original_text=row.original_text,
                                                             rewritten_text=row.rewritten_text,
                                                             rewrite_prompt=row.rewrite_prompt), axis=1)

ori_data.drop(['id', 'original_text', 'rewritten_text','rewrite_prompt'], axis=1, inplace=True)

In [ ]:
ori_data

In [ ]:
from datasets import Dataset
# Load dataset (you can process it here)
dataset = Dataset.from_pandas(ori_data)

In [ ]:
dataset

In [ ]:
urldata = load_dataset(dataset_name, split="train")
urldata['text'][0]

In [ ]:


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


In [ ]:

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="promot",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# !huggingface-cli login

In [ ]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

模型验证

In [ ]:
import pandas as pd
train_data_path='nbroad-v2.csv'
test_data = pd.read_csv(train_data_path, usecols=['original_text', 'rewrite_prompt', 'rewritten_text'])

In [ ]:
def truncate_txt(text, length):
    text_list = text.split()

    if len(text_list) <= length:
        return text

    return " ".join(text_list[:length])


def gen_prompt(og_text, rewritten_text):
    # Truncate the texts to first 200 words for now
    # As we are having memory issues on Mixtral8x7b
    og_text = truncate_txt(og_text,200)
    rewritten_text = truncate_txt(rewritten_text,200)
    promot_tamplate = f"""
<s>[INST] <<SYS>>
    Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.
    You are trying to understand how the original essay was transformed into a new version.
    Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.
    Start directly with the prompt, that's all I need. Output should be only line ONLY.
<</SYS>>
    Original Essay:
    \"""{og_text}\"""

    Rewritten Essay:
    \"""{rewritten_text}\"""
 [/INST]
"""
    return promot_tamplate
    # return f"""
    # Original Essay:
    # \"""{og_text}\"""

    # Rewritten Essay:
    # \"""{rewritten_text}\"""

    # Given are 2 essays, the Rewritten essay was created from the Original essay using the google Gemma model.
    # You are trying to understand how the original essay was transformed into a new version.
    # Analyzing the changes in style, theme, etc., please come up with a prompt that must have been used to guide the transformation from the original to the rewritten essay.
    # Start directly with the prompt, that's all I need. Output should be only line ONLY.
    # """

In [ ]:
test_data=test_data[:5]
test_data["promot"] = test_data.apply(lambda row: gen_prompt(row.original_text,rewritten_text=row.rewritten_text), axis=1)


In [ ]:
test_data

In [ ]:

promot=test_data["promot"][4]
promot

In [ ]:
def clean_memory():
  import gc
  import ctypes
  gc.collect()
  ctypes.CDLL("libc.so.6").malloc_trim(0)
  torch.cuda.empty_cache()
def model_do(promot):
  clean_memory()
  global tokenizer
  global model
  inputs = tokenizer(promot, return_tensors="pt").to('cuda')
# Generate
  generate_ids = model.generate(inputs.input_ids, max_length=1000)
  decoded_output=tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

  # if "Prompt:" in decoded_output:
  #   result = decoded_output[last_prompt_index + len("Prompt:"):].strip()
  # else:
  #   DEFAULT_TEXT = "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."
  #   result = DEFAULT_TEXT
  import re
  decoded_output = re.sub(r"[\s\S]*\[\/INST\]", '', decoded_output, 1)
  return decoded_output


In [ ]:
res = model_do(promot)
res

In [ ]:
test_data["pred"] = test_data.apply(lambda row: model_do(row.promot), axis=1)

In [ ]:
test_data.drop(['original_text','rewritten_text'], axis=1, inplace=True)



In [ ]:
test_data

In [ ]:
test_data.to_csv('output.csv',encoding='utf-8', index=False)


模型打分

In [ ]:
!kaggle datasets download -d yeoyunsianggeremie/sentence-transformers-2-4-0

In [ ]:
!unzip -q sentence-transformers-2-4-0.zip

In [ ]:
!kaggle datasets download -d yeoyunsianggeremie/sentence-t5-base-hf

In [ ]:
!unzip sentence-t5-base-hf.zip

In [ ]:
!pip install -Uq transformers==4.36.0

In [ ]:
!pip install -Uq sentence_transformers-2.4.0-py3-none-any.whl

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
test = pd.read_csv("output.csv")

In [ ]:
def CVScore(test):

    scs = lambda row: abs((cosine_similarity(row["actual_embeddings"], row["pred_embeddings"])) ** 3)

    model = SentenceTransformer('sentence-t5-base')

    test["actual_embeddings"] = test["rewrite_prompt"].progress_apply(lambda x: model.encode(x, normalize_embeddings=True, show_progress_bar=False).reshape(1, -1))
    test["pred_embeddings"] = test["pred"].progress_apply(lambda x: model.encode(x, normalize_embeddings=True, show_progress_bar=False).reshape(1, -1))

    test["score"] = test.apply(scs, axis=1)
    for i in test["score"]:
      print(i)
    return np.mean(test['score'])[0][0]

print(f"CV Score: {CVScore(test)}")